<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/capstone

/content/drive/MyDrive/Github/capstone


In [ ]:
#!git pull   #check that git us up to date
#!git checkout -b shaptonstall # create a new branch

#!git push origin shaptonstall #commit changes to my branch

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@2cd2a19f7b90.(none)')


In [ ]:
# to commit to github
#!git status # check untracked files
#!git add {filename} # add the files
#!git config --global user.email "sarahapt@umich.edu"
# !git commit

hint: Waiting for your editor to close the file... 7[>4;2m="/content/drive/MyDrive/Github/capstone/.git/COMMIT_EDITMSG" 11L, 301B▽  Pzz\[0%m           [>c]10;?]11;?# Please enter the commit message for your changes. Lines starting# with '#' will be ignored, and an empty message aborts the commit.#
# On branch shaptonstall
# Changes to be committed:
#       new file:   data_exploration.ipynb
#
# Changes not staged for commit:
#       modified:   data_exploration.ipynb
#
~                                                                                                   ~                                                                                                   ~                                                                                                   ~                                                                                                   ~                                                                                                   ~              

Import libraries

In [4]:
import pandas as pd

In [5]:
!ls

Capstone.ipynb	data_exploration.ipynb	README.md  Webscraping.xlsx


In [7]:
excel_file = 'Webscraping.xlsx'

# Load all sheets into a dictionary of DataFrames
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)



In [8]:
webscraping.keys()

dict_keys(['Events and8', 'Events WDSF', 'Battles Trivium', 'Battles and8 Threefold', 'Battles and8 Single Slider', 'Research Questions', 'Notes', 'Notes Columns'])

In [29]:
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')
trivium.head(2)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r5j8but2,r5j9over,r5j9tech,r5j9vari,r5j9perf,r5j9musi,r5j9crea,r5j9pers,r5j9but1,r5j9but2
0,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Ami,671,671,3,9,1,2,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Sunny,Anti,Anti,3,9,1,2,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
trivium.describe()

,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,breaker 2 vote count,r1j1over,r1j1tech,r1j1vari,r1j1perf,...,r5j8but2,r5j9over,r5j9tech,r5j9vari,r5j9perf,r5j9musi,r5j9crea,r5j9pers,r5j9but1,r5j9but2
count,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000,1526.000000,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
mean,1.950197,5.989515,0.795544,1.154653,4.897117,6.688729,1.288388,0.322149,0.045020,0.390564,...,NaN,0.8,0.8,0.0,0.0,0.0,0.0,0.0,NaN,NaN
std,0.728895,1.868783,0.950655,0.931595,4.782814,5.024344,7.363129,3.048413,1.021697,2.956864,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,-35.330000,-14.000000,-5.870000,-18.000000,...,NaN,0.8,0.8,0.0,0.0,0.0,0.0,0.0,NaN,NaN
25%,2.000000,5.000000,0.000000,0.000000,1.000000,3.000000,-3.400000,-2.000000,0.000000,-1.600000,...,NaN,0.8,0.8,0.0,0.0,0.0,0.0,0.0,NaN,NaN
50%,2.000000,5.000000,0.000000,1.000000,4.000000,6.000000,1.270000,0.000000,0.000000,0.000000,...,NaN,0.8,0.8,0.0,0.0,0.0,0.0,0.0,NaN,NaN
75%,2.000000,9.000000,2.000000,2.000000,7.000000,9.000000,5.270000,2.200000,0.097500,2.000000,...,NaN,0.8,0.8,0.0,0.0,0.0,0.0,0.0,NaN,NaN
max,5.000000,9.000000,4.000000,4.000000,34.000000,32.000000,49.470000,14.000000,4.800000,20.000000,...,NaN,0.8,0.8,0.0,0.0,0.0,0.0,0.0,NaN,NaN
